In [1]:
import pickle

import os, sys
WORKING_DIR = os.path.dirname(os.path.abspath(''))
sys.path.insert(1, os.path.join(WORKING_DIR,'LensQuEst'))

##### 
import warnings
warnings.filterwarnings("ignore")
#####

from universe import *
from halo_fit import *
from cmb import *
from flat_map import *
from weight import *
from pn_2d import *

In [2]:
oup_fname = '../data/input/universe_Planck15/camb/CAMB_outputs.pkl'
print(oup_fname)
f = open(oup_fname, 'rb') 
powers,cl,c_lensed,c_lens_response = pickle.load(f)
f.close()

totCL=powers['total']
unlensedCL=powers['unlensed_scalar']

L = np.arange(unlensedCL.shape[0])

unlensedTT = unlensedCL[:,0]/(L*(L+1))*2*np.pi
F = unlensedTT
funlensedTT = interp1d(L, F, kind='linear', bounds_error=False, fill_value=0.)

L = np.arange(cl.shape[0])
PP = cl[:,0]
rawPP = PP*2*np.pi/((L*(L+1))**2)
rawKK = L**4/4 * rawPP

fKK = interp1d(L, rawKK, kind='linear', bounds_error=False, fill_value=0.)

L = np.arange(totCL.shape[0])

lensedTT = totCL[:,0]/(L*(L+1))*2*np.pi
F = lensedTT
flensedTT = interp1d(L, F, kind='linear', bounds_error=False, fill_value=0.)


ftot = lambda l : flensedTT(l) + cmb.fForeground(l) + cmb.fdetectorNoise(l)
ftot = lambda l : flensedTT(l) + cmb.fForeground(l) + cmb.fdetectorNoise(l)


with open('f_aniso_ftot_worst_case.pkl', 'rb') as f:
    ftot = pickle.load(f)
print('loaded estimated ftot')



L = np.arange(c_lens_response.shape[0])

cTgradT = c_lens_response.T[0]/(L*(L+1))*2*np.pi

fTgradT = interp1d(L, cTgradT, kind='linear', bounds_error=False, fill_value=0.)

../data/input/universe_Planck15/camb/CAMB_outputs.pkl
loaded estimated ftot


In [3]:



#####
N_runs = 2000

process = True
# number of pixels for the flat map
nX = 1200
nY = 1200
# map dimensions in degrees
sizeX = 20.
sizeY = 20.

#####


print("Map properties")


mean_field = None


# basic map object
baseMap = FlatMap(nX=nX, nY=nY, sizeX=sizeX*np.pi/180., sizeY=sizeY*np.pi/180.)

# multipoles to include in the lensing reconstruction
lMin = 30.; lMax = 3.5e3

# ell bins for power spectra
lRange = (1., 2.*lMax)  # range for power spectra

print("CMB experiment properties")

# Adjust the lMin and lMax to the assumptions of the analysis
# CMB S3 specs
cmb = StageIVCMB(beam=1.4, noise=7., lMin=lMin, lMaxT=lMax, lMaxP=lMax, atm=False)

# Total power spectrum, for the lens reconstruction
# basiscally gets what we theoretically expect the
# power spectrum will look like
forCtotal = lambda l: ftot(l) 

# reinterpolate: gain factor 10 in speed
# L = np.logspace(np.log10(lMin/2.), np.log10(2.*lMax), 1001, 10.)
# F = np.array(list(map(forCtotal, L)))
cmb.fCtotal = ftot

print("Gets a theoretical prediction for the noise")
fNqCmb_fft = baseMap.forecastN0Kappa(funlensedTT, cmb.fCtotal, lMin=lMin, lMax=lMax, test=False)

Map properties
CMB experiment properties
Gets a theoretical prediction for the noise
computing the reconstruction noise


In [4]:
from multiprocessing import Pool
from tqdm import trange, tqdm
errmap = None

with open('anisotropic_noise_map_worst_case.pkl', 'rb') as f:
    errmap = pickle.load(f)

def process_run(i):
    np.random.seed(i+609238472)
    cmb0Fourier = baseMap.genGRF(funlensedTT, test=False)
    cmb0 = baseMap.inverseFourier(cmb0Fourier)

    kCmbFourier = baseMap.genGRF(fKK, test=False)
    kCmb = baseMap.inverseFourier(kCmbFourier)

    lensedCmb = baseMap.doLensing(cmb0, kappaFourier=kCmbFourier)
    lensedCmbFourier = baseMap.fourier(lensedCmb)

    fgFourier = baseMap.genGRF(cmb.fForeground, test=False)
    
    lensedCmbFourier = lensedCmbFourier + fgFourier
    lensedCmb = baseMap.inverseFourier(lensedCmbFourier)

    noiseFourier = baseMap.genGRF(cmb.fdetectorNoise, test=False)
    

    #first we'll renormalize error map so that when we apply the error map, 
    #at best, anisotropic_noise = isotropic noise
#     print('applying anisotropic detector noise')
    aniso_noise = baseMap.inverseFourier(noiseFourier) / np.min(errmap)
    aniso_noise = aniso_noise * errmap # apply errmap
    aniso_noise_fourier = baseMap.fourier(aniso_noise)

 
    totalCmbFourier = lensedCmbFourier + aniso_noise_fourier
    totalCmb = baseMap.inverseFourier(totalCmbFourier)

    kappa_map = baseMap.computeQuadEstKappaNorm(
        fTgradT, cmb.fCtotal, lMin=lMin, lMax=lMax, dataFourier=totalCmbFourier
    )
    return kappa_map


with Pool() as pool:
    results = list(tqdm(pool.imap(process_run, range(N_runs)), total=N_runs))
mean_field = sum(results)

100%|██████████| 2000/2000 [2:30:18<00:00,  4.51s/it]  


In [5]:
f = open('/oak/stanford/orgs/kipac/users/delon/LensQuEst/mean_field_1200x1200_20x20_worst_case_aniso_noise.pkl', 'wb') 
pickle.dump(mean_field/N_runs, f)
f.close()

In [6]:
mean_field.shape

(1200, 601)

In [7]:
np.array(results).shape

(2000, 1200, 601)

In [8]:
# f = open('/oak/stanford/orgs/kipac/users/delon/LensQuEst/mean_field_1200x1200_20x20_aniso_noise.pkl', 'wb') 
# pickle.dump(np.mean(results, axis=0), f)
# f.close()

In [9]:
f = open('/oak/stanford/orgs/kipac/users/delon/LensQuEst/mean_field_1200x1200_20x20_worst_case_aniso_noise_intermediate_0.pkl', 'wb') 
pickle.dump(results, f)
f.close()